In [28]:
# import libraries
from shutil import copyfile
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from xml.etree import ElementTree
import glob
import os.path, time
import datetime
from tqdm import tqdm_notebook
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import random
from collections import Counter
from random import sample, choices
import cv2
from tqdm.notebook import tqdm
import pickle
plt.rcParams["figure.figsize"] = [10,10]

In [2]:
# base path for xml
xml_base_path = "/home/ved/ved/Annotations/Images/"

In [3]:
images_base_path = "/home/ved/ved/Images/Shoes/"

In [4]:
# annotation files
os.chdir(xml_base_path)
annotation_files = os.listdir(xml_base_path )
annotation_file_names =[i.split("\\")[-1].split(".")[0] for i in annotation_files]
annotation_file_names = [i for i in annotation_file_names if len(i) !=0]

In [5]:
annotation_file_names[:5]

['skechers-work-holdredge-black-action-leather-trim_product_8861515_color_680937',
 'lowa-zephyr-gtx-mid-tf-wolf-grey_product_8575700_color_219303',
 'georgia-boot-g5153-12-wellington-barracuda-gold_product_121847_color_584864',
 'nike-free-rn-2018-gym-red-bright-crimson-black-team-red_product_8982513_color_748881',
 'franco-sarto-orchard-black-bally-leather_product_9197825_color_711287']

In [6]:
len(annotation_file_names)

4756

In [7]:
def check_dim(name):
    name = name
    xml_path = xml_base_path + name + ".xml"
    img = plt.imread(images_base_path  +name + ".jpg")
    soup = BeautifulSoup(open(xml_path).read())
    x_dim = img.shape[1]
    y_dim = img.shape[0]
    x_xml = soup.find_all(["x","xmin","xmax"])
    y_xml = soup.find_all(["y","ymin","ymax"])
    x_txt = [float(i.text) for i in x_xml]
    y_txt = [float(i.text) for i in y_xml]
    ind_x = sum([i for i in x_txt if i > x_dim])
    ind_y = sum([i for i in y_txt if i > y_dim])
    if ((ind_x > 0) or (ind_y >0)):
        return "nf"
    else:
        return name

In [8]:
# uncomment to get data
catch_images_shoes = Parallel(n_jobs=47, verbose = 10, backend = "threading")(delayed(check_dim)(file) for file in annotation_file_names)

[Parallel(n_jobs=47)]: Using backend ThreadingBackend with 47 concurrent workers.
[Parallel(n_jobs=47)]: Done   4 tasks      | elapsed:    0.6s
[Parallel(n_jobs=47)]: Done  19 tasks      | elapsed:    1.0s
[Parallel(n_jobs=47)]: Done  34 tasks      | elapsed:    1.5s
[Parallel(n_jobs=47)]: Done  51 tasks      | elapsed:    2.1s
[Parallel(n_jobs=47)]: Done  68 tasks      | elapsed:    2.8s
[Parallel(n_jobs=47)]: Done  87 tasks      | elapsed:    6.8s
[Parallel(n_jobs=47)]: Done 106 tasks      | elapsed:    6.8s
[Parallel(n_jobs=47)]: Done 127 tasks      | elapsed:    6.8s
[Parallel(n_jobs=47)]: Done 148 tasks      | elapsed:    6.8s
[Parallel(n_jobs=47)]: Done 171 tasks      | elapsed:    6.9s
[Parallel(n_jobs=47)]: Done 194 tasks      | elapsed:   11.3s
[Parallel(n_jobs=47)]: Done 219 tasks      | elapsed:   11.3s
[Parallel(n_jobs=47)]: Done 244 tasks      | elapsed:   11.3s
[Parallel(n_jobs=47)]: Done 271 tasks      | elapsed:   11.3s
[Parallel(n_jobs=47)]: Done 298 tasks      | elaps

In [9]:
paths_correct = [i for i in catch_images_shoes if i !="nf"]

In [10]:
len(paths_correct)

1959

In [11]:
len([i for i in paths_correct])/len(annotation_files)

0.41181416859365144

In [12]:
def parse_xmls(name): 
    # read the file name
    xml_path =  xml_base_path + name + ".xml"
    img = plt.imread(images_base_path  +name + ".jpg")
    width = img.shape[1]
    height = img.shape[0]
    # make soup
    # pass the path to the beautiful soup function
    soup = BeautifulSoup(open(xml_path).read())
    soup_obj = soup.find_all("object")
    catch = []
    for item in range(len(soup_obj)):
        class_name = soup_obj[item].find_all("name")[0].text
        x_pts = [int(round(float(i.text),0)) for i in soup_obj[item].find_all(["x","xmin","xmax"])]
        y_pts = [int(round(float(i.text),0)) for i in soup_obj[item].find_all(["y","ymin","ymax"])]
        xmin = np.min(x_pts)
        ymin = np.min(y_pts)
        xmax = np.max(x_pts)
        ymax = np.max(y_pts)
        catch_obj = {"class": class_name, "xmin":xmin, "ymin":ymin,"xmax":xmax,"ymax":ymax}
        catch_obj["filename"] = name + ".jpg"
        catch_obj["width"] = width
        catch_obj["height"] = height
        catch.append(catch_obj)
    return(catch)
        

In [13]:
# uncomment to get data
catch_images_shoes = Parallel(n_jobs=47, verbose = 10, backend = "threading")(delayed(parse_xmls)(file) for file in paths_correct)

[Parallel(n_jobs=47)]: Using backend ThreadingBackend with 47 concurrent workers.
[Parallel(n_jobs=47)]: Done   4 tasks      | elapsed:    1.2s
[Parallel(n_jobs=47)]: Done  19 tasks      | elapsed:    2.2s
[Parallel(n_jobs=47)]: Done  34 tasks      | elapsed:    3.1s
[Parallel(n_jobs=47)]: Done  51 tasks      | elapsed:    4.1s
[Parallel(n_jobs=47)]: Done  68 tasks      | elapsed:    5.4s
[Parallel(n_jobs=47)]: Done  87 tasks      | elapsed:    6.9s
[Parallel(n_jobs=47)]: Done 106 tasks      | elapsed:    8.1s
[Parallel(n_jobs=47)]: Done 127 tasks      | elapsed:    9.5s
[Parallel(n_jobs=47)]: Done 148 tasks      | elapsed:   10.2s
[Parallel(n_jobs=47)]: Done 171 tasks      | elapsed:   12.0s
[Parallel(n_jobs=47)]: Done 194 tasks      | elapsed:   13.6s
[Parallel(n_jobs=47)]: Done 219 tasks      | elapsed:   14.8s
[Parallel(n_jobs=47)]: Done 244 tasks      | elapsed:   16.6s
[Parallel(n_jobs=47)]: Done 271 tasks      | elapsed:   18.1s
[Parallel(n_jobs=47)]: Done 298 tasks      | elaps

In [14]:
# uncomment to get data
catch_images_shoes = [j for i in catch_images_shoes for j in i]

In [15]:
# uncomment to get data
catch_images_shoes_df = pd.DataFrame(catch_images_shoes)

In [16]:
# catch_images_shoes_df = pd.DataFrame(catch_images_shoes)

In [17]:
# catch_images_shoes_df.to_csv("/home/ved/ved/Data/shoe_annotations.csv", index = False)

In [18]:
catch_images_shoes_df.head()

,class,xmin,ymin,xmax,ymax,filename,width,height
0,exclude,274,640,1049,765,skechers-work-holdredge-black-action-leather-t...,1908,777
1,exclude,663,82,1092,194,skechers-work-holdredge-black-action-leather-t...,1908,777
2,quadrilateral,99,276,298,465,skechers-work-holdredge-black-action-leather-t...,1908,777
3,quadrilateral,309,177,497,367,skechers-work-holdredge-black-action-leather-t...,1908,777
4,quadrilateral,312,374,502,557,skechers-work-holdredge-black-action-leather-t...,1908,777


In [19]:
file_names = np.unique(catch_images_shoes_df["filename"])

In [20]:
len(file_names)

1958

In [21]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [22]:
# write a function to get all the region proposals

def get_rpn(filename,method = "quality"): 
    base_path = images_base_path
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
#     base_path = r"D:\Shoes\Shoes_with_annotations\\"
    filename = filename
    samp_path = base_path + filename
    im = cv2.imread(samp_path)
    ss.setBaseImage(im)
    if method == "fast":
        print(method)
        ss.switchToSelectiveSearchFast()
    if method == "quality":
        print(method)
        ss.switchToSelectiveSearchQuality()
    rects = ss.process()
    coords = []
    for (x, y, w, h) in rects: 
        xmin, ymin, xmax, ymax = x,y,x+w, y+h
        coords.append([xmin, ymin, xmax, ymax])
    return(coords)

In [23]:
# write a function to get all the region proposals

def get_rpn_df(filename,method = "fast"):
#     store_path = r"D:\Shoes\Latest_Iteration\Robot_Annotated\\"
    base_path = images_base_path
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
#     base_path = base_path
    filename = filename
    samp_path = base_path + filename
    temp_data = catch_images_shoes_df[catch_images_shoes_df["filename"] == filename]
    temp_data = temp_data[temp_data["class"] != "exclude"]
    im = cv2.imread(samp_path)
    ss.setBaseImage(im)
    if method == "fast":
#         print(method)
        ss.switchToSelectiveSearchFast()
    if method == "quality":
#         print(method)
        ss.switchToSelectiveSearchQuality()
    rects = ss.process()
    coords = []
    for (x, y, w, h) in rects: 
        xmin, ymin, xmax, ymax = x,y,x+w, y+h
        coords.append([xmin, ymin, xmax, ymax])
    robot_dict = []
    for i in coords:
        human_annotated_dict = {"x1": i[0], "x2": i[2], 
                       "y1": i[1],"y2": i[3]}
        robot_dict.append(human_annotated_dict)
    best_catches = []
    max_value_catch = []
    for i in temp_data.values:
        class_name = i[0]
        human_annotated = i[1:5]
        filename = i[5]
        width, height = i[6], i[7]
        human_annotated_dict = {"x1": human_annotated[0], "x2": human_annotated[2], 
                       "y1": human_annotated[1],"y2": human_annotated[3]}
        iou_human_vs_robot = [get_iou(n,human_annotated_dict) for i,n in enumerate(robot_dict)]
    
        suitable_indexes = [i for i,n in enumerate(iou_human_vs_robot) if n > 0.6]
    
        # get those rpns
    
        rpn_values = [robot_dict[i] for i  in suitable_indexes]
    
        iou_values = [iou_human_vs_robot[i] for i in suitable_indexes]
    
#     max_value = np.max(iou_human_vs_robot)
#     max_value_catch.append(max_value)
#     best_rpn = robot_dict[np.argmax(iou_human_vs_robot)]
    
        for i,n in enumerate(rpn_values): 
            best_rpn = n
            xmin, ymin, xmax, ymax = best_rpn["x1"], best_rpn["y1"], best_rpn["x2"], best_rpn["y2"]
            obj = [xmin, ymin, xmax, ymax]
            dict1 = {"human_rpn": human_annotated, "ss_rpn": obj, "iou": iou_values[i], "class":class_name,
                "filename": filename, "width":width, "height":height}
            best_catches.append(dict1)
#     mean_iou = np.mean(max_value_catch)
#     counter = 0
#     output = im.copy()
#     for i1 in range(0, len(best_catches)):
#         counter = counter + 1
#     # clone the original image so we can draw on it
    
    
#     # loop over the current subset of region proposals
#         i = best_catches[i1]
    
#         xmin = i[0]
#         ymin = i[1]
#         xmax = i[2]
#         ymax = i[3]
#     # draw the region proposal bounding box on the image
# #     color = [random.randint(0, 255) for j in range(0, 3)]
#         cv2.rectangle(output, (xmin, ymin), (xmax, ymax), (0,0,255), 3)
#     plt.imsave(store_path + filename,output[:,:,::-1])
    return(best_catches)

In [ ]:
rpn_df = Parallel(n_jobs=47, verbose = 10, backend = "threading")(delayed(get_rpn_df)(file) for file in file_names)

[Parallel(n_jobs=47)]: Using backend ThreadingBackend with 47 concurrent workers.


In [ ]:
with open('/home/ved/ved/Data/bbox_regression_list.pickle', 'wb') as b:
    pickle.dump(rpn_df,b)

In [32]:
# with open('/home/ved/ved/Data/blah.pickle', "rb") as input_file:
#     e = pickle.load(input_file)